### Modelli teorici utilizzati:
* **viscosità**: ... (indicato dal prof)
* **densità**: [Density model for aqueous glycerol solutions](https://link.springer.com/article/10.1007/s00348-018-2527-y) (trovato da noi)

In [108]:
from rberga06.phylab import Datum as d, Measure
import math
import sympy
from typing import Any, Iterable
from itertools import count
sympy.init_printing()

def abs_(x: Measure[float]) -> Measure[float]:
    return d(abs(x.best), x.delta)

def sin(x: Measure[float]) -> Measure[float]:
    return d(math.sin(x.best), abs(math.cos(x.best)) * x.delta)

def exp(x: Measure[float]) -> Measure[float]:
    e_x = math.exp(x.best)
    return d(e_x, e_x * x.delta)

def ln(x: Measure[float]) -> Measure[float]:
    return d(math.log(x.best), x.delta_rel)

def expr_delta(expr: Any, symbols: Iterable[sympy.Symbol], /) -> Any:
    return sympy.sqrt(sum([(expr.diff(x) * sympy.Symbol(f"(\\delta {x})"))**2 for x in symbols], start=sympy.Integer(0)))

def evalf(expr: Any, subs: dict[sympy.Symbol, Measure[float]], /, *, delta: Any = None) -> Measure[float]:
    best_subs = {x: X.best for x, X in subs.items()}
    delta_subs = {sympy.Symbol(f"(\\delta {x})"): X.delta for x, X in subs.items()}
    if delta is None:
        delta = expr_delta(expr, subs)
    return d(expr.evalf(subs=best_subs), delta.evalf(subs=best_subs | delta_subs))

$$\alpha = 1 - x + \frac{abx(1-x)}{ax+b(1-x)}$$
$$(ax+b(1-x))\alpha = (ax+b(1-x))(1-x) + abx(1-x)$$
$$a\alpha x+b\alpha(1-x) = ax(1-x) + b(1-x)^2 + abx(1-x)$$
$$a\alpha x+b\alpha-b\alpha x = ax-ax^2 + b+bx^2-2bx + abx-abx^2$$
$$(a - b + ab) x^2 + (a\alpha-b\alpha - a + 2b - ab) x + b\alpha - b = 0$$

In [109]:
def eta_water(T: Measure[float], /) -> Measure[float]:  # mPa s
    return 1.79 * exp((-1230 - T)*T/(36100 + 360 * T))

def eta_glycerol(T: Measure[float], /) -> Measure[float]:   # mPa s
    return 12100 * exp((-1233 - T)*T/(9900 + 70 * T))

def rho_water(T: Measure[float], /) -> Measure[float]:   # kg / m^3
    return 1000 * (1 - abs_((T - 3.98)/615)**1.71)

def rho_glycerol(T: Measure[float], /) -> Measure[float]:   # kg/m^3
    return (1273 - 0.612 * T)

def _kappa_solution(T: Measure[float], w_g: Measure[float], /) -> Measure[float]:
    A = (1.78e-6 * T**2 - 1.82e-4 * T + 1.41e-2)
    return 1 + A * sin(w_g ** 1.31 * math.pi) ** 0.81

def rho_solution(T: Measure[float], w_g: Measure[float], /) -> Measure[float]:  # g/dm^3
    rho_w = rho_water(T)
    rho_g = rho_glycerol(T)
    # w_g: concentrazione massa/massa
    return _kappa_solution(T, w_g) * (rho_w + (rho_g - rho_w)/(1 + (rho_g/rho_w) * (1/w_g - 1)))


def _solve_alpha2wg() -> tuple[Any, Any, Any, Any]:
    T, alpha, x, a, b = sympy.symbols("T \\alpha x a b")
    equation = (1 - x) + (a * b * x * (1 - x))/(a * x + b * (1 - x)) - alpha
    x1, x2 = sympy.solve(equation, x)
    a_expr = 0.705 - 0.0017 * T
    b_expr = ((4.9 + 0.036 * T) * a ** 2.5).subs({a: a_expr})
    x1 = x1.subs({a: a_expr, b: b_expr})
    x2 = x2.subs({a: a_expr, b: b_expr})
    return x1, expr_delta(x1, [T, alpha]), x2, expr_delta(x2, [T, alpha])
_SOLUTIONS_ALPHA2WG = _solve_alpha2wg()

def alpha2wg(T: Measure[float], alpha: Measure[float]) -> Measure[float]:
    _T, _alpha = sympy.symbols("T \\alpha")
    _x1, _x1delta, _x2, _x2delta = _SOLUTIONS_ALPHA2WG
    x1 = evalf(_x1, {_T: T, _alpha: alpha}, delta=_x1delta)
    x2 = evalf(_x2, {_T: T, _alpha: alpha}, delta=_x2delta)
    # old code
    if False:
        a = 0.705 - 0.0017 * T
        b = (4.9+0.036 * T) * a**2.5
        A = (a - b + a*b)
        B = (a * alpha - b * alpha - a + 2*b - a*b)
        Delta = B**2 - 4*A*(b*alpha - b)
        x1 = (-B + Delta ** .5)/(2*A)
        x2 = (-B - Delta ** .5)/(2*A)
    return min(x1, x2, key=lambda x: x.best)


$$v=\frac{2}{9}(\rho_s - \rho_f) \frac{R^2}{\eta} g$$
$$R^2 = \xi v$$
$$R^2 = \frac{1}{\frac{2}{9}(\rho_s - \rho_f) \frac{g}{\eta}} v$$
$$\xi = \frac{9\eta}{2(\rho_s - \rho_f) g}$$
$$\frac{2(\rho_s - \rho_f) g \xi}{9} = \eta$$

In [6]:
g = d(9.806, 0.01)  # m/s^2
rho_sfere = d(7713, 45)  # kg/m^3

def analysis(T: Measure[float], xi: Measure[float], best_threshold: float = 1e-3) -> Measure[float]:
    print("T =", T, "°C")
    print("ξ =", xi)
    print("η(acqua) =", _eta_water := eta_water(T), "mPa s")
    print("η(glicerina) =", _eta_glycerol := eta_glycerol(T), "mPa s")
    print("⍴(acqua) =", rho_water(T), "kg/m³")
    print("⍴(glicerina) =", _rho_glycerol := rho_glycerol(T), "kg/m³")
    ln_eta_water = ln(_eta_water)
    ln_eta_glycerol = ln(_eta_glycerol)
    ln_diff = (ln_eta_water - ln_eta_glycerol)
    rho_sol = _rho_glycerol  # kg/m^3
    last_rho_sol_best = rho_sol.best
    for i in count(1):
        print(f"Iterazione {i}:")
        print(" con ⍴ =", rho_sol, "kg/m³")
        eta_sol = (2/9) * (rho_sfere - rho_sol) * g * xi * 10**3   # mPa s
        print(" -> η =", eta_sol, "mPa s")
        alpha = (ln(eta_sol) - ln_eta_glycerol)/ln_diff
        print(" -> ⍺ =", alpha)
        cm = alpha2wg(T, alpha)
        print(f" -> c = {cm*100}% m/s")
        rho_sol = rho_solution(T, cm)
        print(" -> ⍴ =", rho_sol, "kg/m³")
        if abs(rho_sol.best - last_rho_sol_best) <= best_threshold:
            break
        last_rho_sol_best = rho_sol.best
    return rho_sol


In [7]:
# NOTA: T è in °C
# T1_piccole = d(24.8, 0.2)
T1 = d(24.6, 0.2)   # °C
xi1 = d(1.822967460e-5, 6.6444977398e-8)
print("--- Giorno 1 ---")
rho1 = analysis(T1, xi1, best_threshold=1e-10)

--- Giorno 1 ---
T = 24.6 ± 0.2 °C
ξ = 1.82296746e-05 ± 6.6444977398e-08
η(acqua) = 0.9009516878253878 ± 0.006117825638836887 mPa s
η(glicerina) = 844.7408641908731 ± 21.347989296290336 mPa s
⍴(acqua) = 996.9907576701406 ± 0.04991080876876254 kg/m³
⍴(glicerina) = 1257.9448 ± 0.12240000000000001 kg/m³
Iterazione 1:
 con ⍴ = 1257.9448 ± 0.12240000000000001 kg/m³
 -> η = 256.4237529735773 ± 2.988594782528412 mPa s
 -> ⍺ = 0.17421312876759923 ± 0.0062122031543937655


 -> c = 93.5605863290863 ± 0.260710918069600% m/s
 -> ⍴ = 1241.53018353832 ± 1.20820559239222 kg/m³
Iterazione 2:
 con ⍴ = 1241.53018353832 ± 1.20820559239222 kg/m³
 -> η = 257.075815183165 ± 3.03476950312097 mPa s
 -> ⍺ = 0.1738420112376056 ± 0.00623239129942674
 -> c = 93.5761566997101 ± 0.261407529070465% m/s
 -> ⍴ = 1241.57154122762 ± 1.21095214014362 kg/m³
Iterazione 3:
 con ⍴ = 1241.57154122762 ± 1.21095214014362 kg/m³
 -> η = 257.074172270196 ± 3.03487094468322 mPa s
 -> ⍺ = 0.17384294510887677 ± 0.00623246436111109
 -> c = 93.5761175300572 ± 0.261410972399767% m/s
 -> ⍴ = 1241.57143718886 ± 1.21096464005990 kg/m³
Iterazione 4:
 con ⍴ = 1241.57143718886 ± 1.21096464005990 kg/m³
 -> η = 257.074176403082 ± 3.03487146051449 mPa s
 -> ⍺ = 0.1738429427596375 ± 0.00623246461558261
 -> c = 93.5761176285922 ± 0.261410982119996% m/s
 -> ⍴ = 1241.57143745058 ± 1.21096467728833 kg/m³
Iterazione 5:
 con ⍴ = 1241.57143745058 ± 1.21096467728833 kg/m³
 -> η = 257.074176392686 ± 3.0348714619448

In [8]:
xi2 = d(2.6374145872e-5, 9.5797126107e-8)
T2 = d(19.4, 0.2)   # °C
print("--- Giorno 2 ---")
rho2 = analysis(T2, xi2, best_threshold=1e-10)

--- Giorno 2 ---
T = 19.4 ± 0.2 °C
ξ = 2.6374145872e-05 ± 9.5797126107e-08
η(acqua) = 1.0198258016822541 ± 0.006965464972506146 mPa s
η(glicerina) = 1398.0045823591865 ± 35.33807802114838 mPa s
⍴(acqua) = 998.1691707971398 ± 0.04060593951868824 kg/m³
⍴(glicerina) = 1261.1272 ± 0.12240000000000001 kg/m³
Iterazione 1:
 con ⍴ = 1261.1272 ± 0.12240000000000001 kg/m³
 -> η = 370.8032872620404 ± 4.318267633461683 mPa s
 -> ⍺ = 0.18373229721313852 ± 0.005928480585016183


 -> c = 93.1710194049245 ± 0.251638726135034% m/s
 -> ⍴ = 1243.72560825954 ± 1.16558273120656 kg/m³
Iterazione 2:
 con ⍴ = 1243.72560825954 ± 1.16558273120656 kg/m³
 -> η = 371.803394923951 ± 4.38287415489932 mPa s
 -> ⍺ = 0.18335939894920958 ± 0.00594654285379272
 -> c = 93.1868427704386 ± 0.252260015162105% m/s
 -> ⍴ = 1243.76770258301 ± 1.16807431983722 kg/m³
Iterazione 3:
 con ⍴ = 1243.76770258301 ± 1.16807431983722 kg/m³
 -> η = 371.800975670813 ± 4.38300609755138 mPa s
 -> ⍺ = 0.18336029977658705 ± 0.00594660660717897
 -> c = 93.1868045561943 ± 0.252263070700363% m/s
 -> ⍴ = 1243.76760092591 ± 1.16808551135548 kg/m³
Iterazione 4:
 con ⍴ = 1243.76760092591 ± 1.16808551135548 kg/m³
 -> η = 371.800981513270 ± 4.38300676793171 mPa s
 -> ⍺ = 0.18336029760109662 ± 0.00594660682148484
 -> c = 93.1868046484814 ± 0.252263078943720% m/s
 -> ⍴ = 1243.76760117141 ± 1.16808554334088 kg/m³
Iterazione 5:
 con ⍴ = 1243.76760117141 ± 1.16808554334088 kg/m³
 -> η = 371.800981499160 ± 4.38300676970

In [120]:
print(rho2.ε(rho1))

0.923126255004128
